# Интеллектуальные методы обработки видео

## Модуль 5


In [1]:
# Константы
dataFolder = "./data"

## Стерео видео

https://docs.luxonis.com/en/latest/

https://github.com/isl-org/MiDaS


In [2]:
from PIL import Image
import cv2
import torch
import numpy as np
import torchvision.transforms as T
from PIL import Image

# Загрузка модели MiDaS для оценки глубины из PyTorch Hub
model = torch.hub.load("intel-isl/MiDaS", "MiDaS")
model.eval()

# Если у вас есть GPU, разкомментируйте следующую строку для ускорения
# model.to(torch.device("cuda"))

from torchvision.transforms.functional import pad


def resize_and_pad(img, base=32):
    """Resize the image to a size that is a multiple of `base`, then pad if necessary."""
    w, h = img.size
    new_w = ((w - 1) // base + 1) * base
    new_h = ((h - 1) // base + 1) * base
    resized_img = T.Resize((new_h, new_w))(img)
    # Calculate padding
    pad_left = pad_top = 0
    pad_right = new_w - w
    pad_bottom = new_h - h
    padded_img = pad(resized_img, (pad_left, pad_top, pad_right, pad_bottom))
    return padded_img


# Обработка кадра для совместимости с моделью MiDaS
def process_frame(frame, model):
    # Convert the color from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)

    # Custom resize to ensure dimension compatibility
    desired_size = 384  # Example size, adjust as necessary
    frame_resized = frame_pil.resize((desired_size, desired_size), Image.BICUBIC)

    # Prepare the transforms
    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    input_tensor = transform(frame_resized).unsqueeze(0)  # Add batch dimension

    # Move input data to the GPU if available
    input_tensor = input_tensor.to(next(model.parameters()).device)

    with torch.no_grad():
        # Model prediction
        prediction = model(input_tensor)

        # Resize prediction to original frame size
        depth_map = torch.nn.functional.interpolate(prediction.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False).squeeze()
        depth_map = depth_map.cpu().numpy()

    # Normalize depth map for visualization
    depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map_normalized = depth_map_normalized.astype(np.uint8)
    return depth_map_normalized


# Чтение видео
cap = cv2.VideoCapture("./data/src/Mating.mp4")
frame_count = 0

# while True:
for i in range(10):
    ret, frame = cap.read()
    if not ret:
        break

    # Получение и сохранение карты глубины
    depth_map = process_frame(frame, model)
    cv2.imwrite(f"./data/result/frame_{frame_count:04d}.png", depth_map)

    frame_count += 1

cap.release()

Using cache found in C:\Users\inimatic/.cache\torch\hub\intel-isl_MiDaS_master
c:\git\MUIV\video_pocessing\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading weights:  None


Using cache found in C:\Users\inimatic/.cache\torch\hub\facebookresearch_WSL-Images_main


In [3]:
from PIL import Image
import cv2
import torch
import numpy as np
import torchvision.transforms as T
from torchvision.transforms.functional import pad

# Загрузка модели MiDaS для оценки глубины из PyTorch Hub
model = torch.hub.load("intel-isl/MiDaS", "MiDaS")
model.eval()

# Перенос модели на GPU для ускорения, если доступно
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


def process_frame(frame, model):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)

    # Подготовка изображения
    frame_resized = resize_and_pad(frame_pil)

    # Преобразование и нормализация кадра
    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    input_tensor = transform(frame_resized).unsqueeze(0)
    input_tensor = input_tensor.to(next(model.parameters()).device)

    with torch.no_grad():
        prediction = model(input_tensor)
        depth_map = torch.nn.functional.interpolate(prediction.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False).squeeze()
        depth_map = depth_map.cpu().numpy()

    # Нормализация карты глубины для визуализации
    depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map_normalized = depth_map_normalized.astype(np.uint8)
    return depth_map_normalized


def resize_and_pad(img, base=32, desired_size=384):
    # Изменение размера изображения с учетом базового размера
    w, h = img.size
    new_w, new_h = desired_size, desired_size
    resized_img = T.Resize((new_h, new_w))(img)
    return resized_img


# Чтение исходного видео
cap = cv2.VideoCapture(f"{dataFolder}/src/Mating.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(f"{dataFolder}/result/3d_video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 20.0, (frame_width, frame_height))

# while True:
for i in range(10):
    ret, frame = cap.read()
    if not ret:
        break

    # Получение карты глубины для текущего кадра
    depth_map = process_frame(frame, model)

    # Здесь можно добавить код для создания 3D эффекта, используя карту глубины
    # В этом примере мы просто сохраним оригинальный кадр
    # Для демонстрации мы используем карту глубины как часть видео
    # Чтобы видео не было статичным, добавим эффект параллакса
    frame_with_depth = cv2.applyColorMap(depth_map, cv2.COLORMAP_JET)
    out.write(frame_with_depth)

cap.release()
out.release()

Using cache found in C:\Users\inimatic/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\inimatic/.cache\torch\hub\facebookresearch_WSL-Images_main


In [4]:
import cv2
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as T

# Загрузка модели MiDaS для оценки глубины из PyTorch Hub
model = torch.hub.load("intel-isl/MiDaS", "MiDaS")
model.eval()
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


def resize_and_pad(img, base=32, desired_size=384):
    w, h = img.size
    new_w, new_h = desired_size, desired_size
    resized_img = T.Resize((new_h, new_w))(img)
    return resized_img


def process_frame(frame, model):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    frame_resized = resize_and_pad(frame_pil)
    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    input_tensor = transform(frame_resized).unsqueeze(0)
    input_tensor = input_tensor.to(next(model.parameters()).device)
    with torch.no_grad():
        prediction = model(input_tensor)
        depth_map = torch.nn.functional.interpolate(prediction.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False).squeeze()
        depth_map = depth_map.cpu().numpy()
    depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map_normalized = depth_map_normalized.astype(np.uint8)
    return depth_map_normalized


cap = cv2.VideoCapture(f"{dataFolder}/src/Mating.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(f"{dataFolder}/result/stereo_video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 20.0, (frame_width * 2, frame_height))


def create_stereo_frame(left_frame, shift):
    # Для простоты, сдвигаем весь кадр влево для левого глаза и вправо для правого глаза
    # Этот метод не учитывает реальную глубину объектов
    right_frame = np.roll(left_frame, shift, axis=1)
    left_frame = np.roll(left_frame, -shift, axis=1)
    return np.concatenate((left_frame, right_frame), axis=1)


while True:
    ret, frame = cap.read()
    if not ret:
        break

    depth_map = process_frame(frame, model)
    # Используйте карту глубины, чтобы определить, как сильно сдвигать пиксели
    # Для примера, используем фиксированный сдвиг
    shift = 30  # Фиксированное значение сдвига для демонстрации
    stereo_frame = create_stereo_frame(frame, shift)

    out.write(stereo_frame)

cap.release()
out.release()

Using cache found in C:\Users\inimatic/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\inimatic/.cache\torch\hub\facebookresearch_WSL-Images_main


In [5]:
# Упрощенный вариант с простым смещением пикселов
import cv2
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as T

# Загрузка модели MiDaS для оценки глубины из PyTorch Hub
model = torch.hub.load("intel-isl/MiDaS", "MiDaS")
model.eval()
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


def resize_and_pad(img, base=32, desired_size=384):
    w, h = img.size
    new_w, new_h = desired_size, desired_size
    resized_img = T.Resize((new_h, new_w))(img)
    return resized_img


def process_frame(frame, model):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    frame_resized = resize_and_pad(frame_pil)
    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    input_tensor = transform(frame_resized).unsqueeze(0)
    input_tensor = input_tensor.to(next(model.parameters()).device)
    with torch.no_grad():
        prediction = model(input_tensor)
        depth_map = torch.nn.functional.interpolate(prediction.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False).squeeze()
        depth_map = depth_map.cpu().numpy()
    depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map_normalized = depth_map_normalized.astype(np.uint8)
    return depth_map_normalized


def create_stereo_frame(left_frame, depth_map):
    max_shift = 30  # Максимальный сдвиг для объектов на минимальной глубине
    depth_map_scaled = (255 - depth_map) / 255  # Инвертирование и масштабирование карты глубины
    shift_map = (depth_map_scaled * max_shift).astype(np.int32)

    height, width = left_frame.shape[:2]
    right_frame = np.zeros_like(left_frame)

    for y in range(height):
        for x in range(width):
            shift = shift_map[y, x]
            new_x = x + shift if x + shift < width else x - shift
            right_frame[y, new_x] = left_frame[y, x]

    left_frame_shifted = np.roll(left_frame, -max_shift // 2, axis=1)  # Простой сдвиг всего изображения
    stereo_frame = np.concatenate((left_frame_shifted, right_frame), axis=1)
    return stereo_frame


cap = cv2.VideoCapture(f"{dataFolder}/src/Mating.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(f"{dataFolder}/result/stereo_video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 20.0, (frame_width * 2, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    depth_map = process_frame(frame, model)
    stereo_frame = create_stereo_frame(frame, depth_map)
    out.write(stereo_frame)

cap.release()
out.release()

Using cache found in C:\Users\inimatic/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\inimatic/.cache\torch\hub\facebookresearch_WSL-Images_main


В этой модификации функции create_stereo_frame добавлена дополнительная функция fill_holes, которая пробегается по всем пикселям в смещенном правом изображении и, если находит "дырку" (пиксель, равный нулю), заполняет её значением ближайшего ненулевого пикселя. Этот подход довольно простой и может быть не идеален для всех сценариев, поскольку в реальности текстуры и детали объектов могут существенно отличаться, и более сложные методы интерполяции могут дать лучший визуальный результат.


In [6]:
import cv2
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as T

# Загрузка модели MiDaS для оценки глубины из PyTorch Hub
model = torch.hub.load("intel-isl/MiDaS", "MiDaS")
model.eval()
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


def resize_and_pad(img, base=32, desired_size=384):
    w, h = img.size
    new_w, new_h = desired_size, desired_size
    resized_img = T.Resize((new_h, new_w))(img)
    return resized_img


def process_frame(frame, model):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    frame_resized = resize_and_pad(frame_pil)
    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    input_tensor = transform(frame_resized).unsqueeze(0)
    input_tensor = input_tensor.to(next(model.parameters()).device)
    with torch.no_grad():
        prediction = model(input_tensor)
        depth_map = torch.nn.functional.interpolate(prediction.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False).squeeze()
        depth_map = depth_map.cpu().numpy()
    depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map_normalized = depth_map_normalized.astype(np.uint8)
    return depth_map_normalized


def fill_holes(interpolated_frame, depth_map):
    h, w = depth_map.shape
    for y in range(h):
        for x in range(w):
            if interpolated_frame[y, x].sum() == 0:  # Проверяем, является ли пиксель "дыркой"
                # Ищем ближайший ненулевой пиксель
                nx, ny = x, y
                while nx < w - 1 and interpolated_frame[ny, nx].sum() == 0:
                    nx += 1
                if interpolated_frame[ny, nx].sum() != 0:
                    interpolated_frame[y, x] = interpolated_frame[ny, nx]
                else:
                    nx, ny = x, y
                    while nx > 0 and interpolated_frame[ny, nx].sum() == 0:
                        nx -= 1
                    if interpolated_frame[ny, nx].sum() != 0:
                        interpolated_frame[y, x] = interpolated_frame[ny, nx]
    return interpolated_frame


def create_stereo_frame(left_frame, depth_map):
    max_shift = 30  # Максимальный сдвиг
    depth_map_scaled = (255 - depth_map) / 255  # Инвертируем и масштабируем карту глубины
    shift_map = (depth_map_scaled * max_shift).astype(np.int32)

    height, width = left_frame.shape[:2]
    right_frame = np.zeros_like(left_frame)

    for y in range(height):
        for x in range(width):
            shift = shift_map[y, x]
            new_x = x + shift if x + shift < width else width - 1
            right_frame[y, new_x] = left_frame[y, x]

    # Заполнение дырок интерполяцией ближайшего соседа
    right_frame_interpolated = fill_holes(right_frame, depth_map)

    left_frame_shifted = np.roll(left_frame, -max_shift // 2, axis=1)
    stereo_frame = np.concatenate((left_frame_shifted, right_frame_interpolated), axis=1)
    return stereo_frame


cap = cv2.VideoCapture(f"{dataFolder}/src/Mating.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(f"{dataFolder}/result/stereo_video2.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 20.0, (frame_width * 2, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    depth_map = process_frame(frame, model)
    stereo_frame = create_stereo_frame(frame, depth_map)
    out.write(stereo_frame)

cap.release()
out.release()

Using cache found in C:\Users\inimatic/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\inimatic/.cache\torch\hub\facebookresearch_WSL-Images_main
